<a href="https://colab.research.google.com/github/nxxk23/AI-Engineer/blob/main/extract/EasyOCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **START**

In [1]:
!nvidia-smi #using GPU is faster

Mon Sep  2 06:31:10 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pwd

/content


In [4]:
!sudo apt-get install ghostscript
!pip install transformers pythainlp python-crfsuite opencv-python easyocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 libgs9 libgs9-common
  libidn12 libijs-0.35 libjbig2dec0 poppler-data
Suggested packages:
  fonts-noto fonts-freefont-otf | fonts-freefont-ttf fonts-texgyre
  ghostscript-x poppler-utils fonts-japanese-mincho | fonts-ipafont-mincho
  fonts-japanese-gothic | fonts-ipafont-gothic fonts-arphic-ukai
  fonts-arphic-uming fonts-nanum
The following NEW packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 ghostscript libgs9
  libgs9-common libidn12 libijs-0.35 libjbig2dec0 poppler-data
0 upgraded, 10 newly installed, 0 to remove and 49 not upgraded.
Need to get 16.7 MB of archives.
After this operation, 63.0 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 fonts-droid-fallback all 1:6.0.1r16-1.1build1 [1,805 kB]


In [ ]:
from pythainlp.phayathaibert.core import NamedEntityTagger
import re
import pandas as pd

tagger = NamedEntityTagger()

# **MULTIPLE PDFs**

In [27]:
import os
import easyocr
import pandas as pd
import shutil
from pythainlp.phayathaibert.core import NamedEntityTagger
import re
from tempfile import TemporaryDirectory

# Define the function to extract text from images
def extract_text_from_image(image_path):
    reader = easyocr.Reader(['th', 'en'])
    result = reader.readtext(image_path)
    sorted_data = sorted(result, key=lambda x: x[0][0][1])
    plain_text = "\n".join([text for _, text, _ in sorted_data])
    return plain_text

def convert_pdf_to_images(pdf_path, output_dir):
    pdf_filename = os.path.splitext(os.path.basename(pdf_path))[0]
    output_pattern = os.path.join(output_dir, f"{pdf_filename}_%d.png")
    os.system(f'gs -dBATCH -dNOPAUSE -sDEVICE=pngalpha -r300 -sOutputFile="{output_pattern}" "{pdf_path}"')

# Define the function to process and clean multiple PDFs
def process_pdfs_in_directory(pdf_dir):
    tagger = NamedEntityTagger()
    unwanted_terms = [
        'ที่อยู่', 'โทรศัพท์', 'อีเมล', 'linkedin', ':', ',', '-', '|',
        'ประวัติส่วนตัว', 'เกี่ยวกับฉัน', 'about me', 'ชื่อ', 'สกุล', 'tell', 'โทร', 'โทรงาน',
        'ชื่อเล่น', 'อายุ', 'วันเกิด', 'พุทธ', 'ศาสนา', 'สัญชาติ', 'phone',
        'ช่องทางการติดต่อ', '_', 're sume', 'resume', 'resu me'
    ]
    unwanted_pattern = '|'.join(map(re.escape, unwanted_terms))

    def tag_and_clean_text(text):
        ner = tagger.get_ner(text, tag=True)
        pattern = r'<(?!ORGANIZATION|PERCENT|TIME)[^>]+>[^<]+</[^>]+>'
        cleaned_ner = re.sub(pattern, '', ner)
        cleaned_ner = re.sub(r'</?(ORGANIZATION|PERCENT|TIME)>', '', cleaned_ner)
        cleaned_ner = re.sub(unwanted_pattern, '', cleaned_ner)
        cleaned_ner = re.sub(r'\bal\b', 'ai', cleaned_ner, flags=re.IGNORECASE)
        return ner.strip(), cleaned_ner.strip()

    results = []
    for pdf_file in os.listdir(pdf_dir):
        if pdf_file.lower().endswith('.pdf'):
            pdf_path = os.path.join(pdf_dir, pdf_file)

            with TemporaryDirectory() as temp_dir:
                images_dir = os.path.join(temp_dir, 'images')
                os.makedirs(images_dir, exist_ok=True)

                # Convert the PDF to images
                convert_pdf_to_images(pdf_path, images_dir)

                # Extract text from all images related to the PDF file
                raw_text = ""
                pdf_filename = os.path.splitext(os.path.basename(pdf_path))[0]
                for img_file in os.listdir(images_dir):
                    if img_file.startswith(pdf_filename):
                        img_path = os.path.join(images_dir, img_file)
                        raw_text += extract_text_from_image(img_path) + "\n"

                # Tag and clean the extracted text
                tagged_text, cleaned_text = tag_and_clean_text(raw_text)

                # Append the result to the list
                results.append({
                    "PDF File": pdf_file,
                    "Raw_Text": raw_text,
                    "Tagged_Text": tagged_text,
                    "Cleaned_Text": cleaned_text
                })

    # Create a DataFrame with all results
    df = pd.DataFrame(results)
    return df

> 12 pdf take 2.24 mins

In [28]:
# Example usage
pdf_dir = '/content/drive/MyDrive/AIEngineer/resume/resume_LLM'
df = process_pdfs_in_directory(pdf_dir)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/easyocr/detection.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub

In [29]:
df

,PDF File,Raw_Text,Tagged_Text,Cleaned_Text
0,Resume BA 2.pdf,พิมพ์นารา วัฒนชัย\nที่อยู่:\n789 ถนนรัชดาภิเษก...,<PERSON>พ</PERSON><PERSON>พิมพ์</PERSON><PERSO...,เป้าหมายในการทำงาน\nนำความเชี่ยวชาญด้านการวิเค...
1,Resume BA 3.pdf,จิตรลดา\n086-234-5678\n jitlada.s@domain.com\...,<PERSON> </PERSON><PERSON>จิตร</PERSON><PERSON...,เสริม\n business analyst\nเป้าหมายในการทำงาน\n...
2,Resume ba 5.pdf,081-456-7890\nนัทธมน ฺรัตนโชติ\n nattamon. rad...,<PHONE>08</PHONE><PERSON>1-4</PERSON><PERSON>5...,เป้าหมายในการ\n มุ่งมันในการนำทักษะด้านการจัดก...
3,Resume ba2.pdf,ศุภกิจฺ มงคลชัย\n089-234-5678\n supakit.m@dom...,<PERSON> </PERSON><PERSON>ศุภ</PERSON><PERSON>...,ประสบการณ์การ\nเป้าหมายในการทำงาน\nfinancial a...
4,Resume baa.pdf,"ธนกร อินทรีย์พงษ์\n ที่อยู่:\n456 ถนนพระราม 3,...",<PERSON>ธ</PERSON><PERSON>ธน</PERSON><PERSON>ก...,เป้าหมายในการทำงาน\nมุ่งมันที่จะใช้ความรู้และท...
5,resume 6.pdf,linkedin:\nนภัสสร วิวัฒนาวงศ์\nlinkedin.com i...,linkedin:<PERSON>\n</PERSON><PERSON>น</PERSON>...,in\n\n วิศวกรปัญญาประดิษฐ์\n\n ประสบการณ์การทำ...
6,resume 7.pdf,ปัญญา วิริยะชัย\n วิศวกรปัญญาประดิษฐ์\nประสบกา...,<PERSON>ป</PERSON><PERSON>ปัญญา วิริยะชัย</PER...,วิศวกรปัญญาประดิษฐ์\nประสบการณ์การฝึกงาน\nบัณฑ...
7,resume 2.pdf,วงศ์เจริญ\nณ็ชพล\n วิศวกรปัญญาประดิษฐ์\n ประว...,<PERSON> วงศ์เจริญ\nณ็ชพล</PERSON>\n วิศวกรปัญ...,วิศวกรปัญญาประดิษฐ์\n ประวัติการศึกษา\nมหาวิทย...
8,resume 3.pdf,re sume\n ประวัติส่วนตัว\n090-123-4567\npimcha...,re sume\n ประวัติส่วนตัว<PHONE>\n09</PHONE><PH...,ผู้ เชี่ยวชาญด้าน ai ที่มีประสบการณ์ 4 เขี\nใน...
9,resume 4.pdf,อาทิตย์\nรัตนวิจิตร\n วิศวกรปัญญาประดิษฐ์\nปร...,<PERSON> </PERSON><PERSON>อาทิตย์\nรัตนวิจิตร<...,วิศวกรปัญญาประดิษฐ์\nประสบการณํ ทำงาน\nวิศวกร ...


# **SINGLE PDF**

In [19]:
import os
import easyocr
import pandas as pd
import shutil
from pythainlp.phayathaibert.core import NamedEntityTagger
import re
from tempfile import TemporaryDirectory

# Define the function to extract text from images
def extract_text_from_image(image_path):
    reader = easyocr.Reader(['th', 'en'])
    result = reader.readtext(image_path)
    sorted_data = sorted(result, key=lambda x: x[0][0][1])
    plain_text = "\n".join([text for _, text, _ in sorted_data])
    return plain_text

def convert_pdf_to_images(pdf_path, output_dir):
    pdf_filename = os.path.splitext(os.path.basename(pdf_path))[0]
    output_pattern = os.path.join(output_dir, f"{pdf_filename}_%d.png")
    os.system(f'gs -dBATCH -dNOPAUSE -sDEVICE=pngalpha -r300 -sOutputFile="{output_pattern}" "{pdf_path}"')

# Define the function to process and clean a single PDF
def process_single_pdf(pdf_path):
    with TemporaryDirectory() as temp_dir:
        # Create a temporary directory for images
        images_dir = os.path.join(temp_dir, 'images')
        os.makedirs(images_dir, exist_ok=True)

        # Convert the PDF to images in the temporary directory
        convert_pdf_to_images(pdf_path, images_dir)

        # Initialize the Named Entity Tagger
        tagger = NamedEntityTagger()
        unwanted_terms = [
            'ที่อยู่', 'โทรศัพท์', 'อีเมล', 'linkedin', ':', ',', '-', '|',
            'ประวัติส่วนตัว', 'เกี่ยวกับฉัน', 'about me', 'ชื่อ', 'สกุล', 'tell', 'โทร', 'โทรงาน',
            'ชื่อเล่น', 'อายุ', 'วันเกิด', 'พุทธ', 'ศาสนา', 'สัญชาติ', 'phone',
            'ช่องทางการติดต่อ', '_', 're sume', 'resume', 'resu me'
        ]
        unwanted_pattern = '|'.join(map(re.escape, unwanted_terms))

        def tag_and_clean_text(text):
            ner = tagger.get_ner(text, tag=True)
            pattern = r'<(?!ORGANIZATION|PERCENT|TIME)[^>]+>[^<]+</[^>]+>'
            cleaned_ner = re.sub(pattern, '', ner)
            cleaned_ner = re.sub(r'</?(ORGANIZATION|PERCENT|TIME)>', '', cleaned_ner)
            cleaned_ner = re.sub(unwanted_pattern, '', cleaned_ner)
            cleaned_ner = re.sub(r'\bal\b', 'ai', cleaned_ner, flags=re.IGNORECASE)
            return ner.strip(), cleaned_ner.strip()

        # Extract text from all images related to the PDF file
        raw_text = ""
        pdf_filename = os.path.splitext(os.path.basename(pdf_path))[0]
        for img_file in os.listdir(images_dir):
            if img_file.startswith(pdf_filename):
                img_path = os.path.join(images_dir, img_file)
                raw_text += extract_text_from_image(img_path) + "\n"

        # Tag and clean the extracted text
        tagged_text, cleaned_text = tag_and_clean_text(raw_text)

        # Create a DataFrame with the result
        data = {
            "PDF File": os.path.basename(pdf_path),
            "Raw_Text": raw_text,
            "Tagged_Text": tagged_text,
            "Cleaned_Text": cleaned_text
        }
        df = pd.DataFrame([data])

        return df

In [20]:
# Example usage
test = '/content/drive/MyDrive/AIEngineer/resume/test Resume.pdf'
test1 = process_single_pdf(test)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/easyocr/detection.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub

In [21]:
test1

,PDF File,Raw_Text,Tagged_Text,Cleaned_Text
0,test Resume.pdf,คลาวเดีย อัลวิส\n ux/ ui designer\n123-456-789...,<PERSON>ค</PERSON><PERSON>ค</PERSON><PERSON>ลา...,ux/ ui designer\nin\n \nการศึกษา\nฉันเป็นคนที่...


In [22]:
print(test1['Cleaned_Text'].iloc[0])

ux/ ui designer
in
 
การศึกษา
ฉันเป็นคนที่ชอบเรียนรู้และมี
studio shodwe university
2023
ความกระตือรือรันในการทำงาน
ฉันเว่าการเรียนรู้ไม่มีที่สิ้นสุด
คณะสถาปัตยกรรมศาสตร์และการ
และทุกวันคือโอกาสที่จะพัฒนา
 ออกแบบ ภาควิชานิเทศศิลป์
 เกรดเฉลีย 3.7
ตนเอง เราอาจพบกันในห้องเรียน
หรือในการทำงานเป็นทีม ทั่้งนี้
เพื่อแลกเปลี่ยนความรู้และ
ประสบการณ์การทำงาน
 ประสบการณ์ที่มีคุณค่า
2023


# LLM PROMPT

In [30]:
df

,PDF File,Raw_Text,Tagged_Text,Cleaned_Text
0,Resume BA 2.pdf,พิมพ์นารา วัฒนชัย\nที่อยู่:\n789 ถนนรัชดาภิเษก...,<PERSON>พ</PERSON><PERSON>พิมพ์</PERSON><PERSO...,เป้าหมายในการทำงาน\nนำความเชี่ยวชาญด้านการวิเค...
1,Resume BA 3.pdf,จิตรลดา\n086-234-5678\n jitlada.s@domain.com\...,<PERSON> </PERSON><PERSON>จิตร</PERSON><PERSON...,เสริม\n business analyst\nเป้าหมายในการทำงาน\n...
2,Resume ba 5.pdf,081-456-7890\nนัทธมน ฺรัตนโชติ\n nattamon. rad...,<PHONE>08</PHONE><PERSON>1-4</PERSON><PERSON>5...,เป้าหมายในการ\n มุ่งมันในการนำทักษะด้านการจัดก...
3,Resume ba2.pdf,ศุภกิจฺ มงคลชัย\n089-234-5678\n supakit.m@dom...,<PERSON> </PERSON><PERSON>ศุภ</PERSON><PERSON>...,ประสบการณ์การ\nเป้าหมายในการทำงาน\nfinancial a...
4,Resume baa.pdf,"ธนกร อินทรีย์พงษ์\n ที่อยู่:\n456 ถนนพระราม 3,...",<PERSON>ธ</PERSON><PERSON>ธน</PERSON><PERSON>ก...,เป้าหมายในการทำงาน\nมุ่งมันที่จะใช้ความรู้และท...
5,resume 6.pdf,linkedin:\nนภัสสร วิวัฒนาวงศ์\nlinkedin.com i...,linkedin:<PERSON>\n</PERSON><PERSON>น</PERSON>...,in\n\n วิศวกรปัญญาประดิษฐ์\n\n ประสบการณ์การทำ...
6,resume 7.pdf,ปัญญา วิริยะชัย\n วิศวกรปัญญาประดิษฐ์\nประสบกา...,<PERSON>ป</PERSON><PERSON>ปัญญา วิริยะชัย</PER...,วิศวกรปัญญาประดิษฐ์\nประสบการณ์การฝึกงาน\nบัณฑ...
7,resume 2.pdf,วงศ์เจริญ\nณ็ชพล\n วิศวกรปัญญาประดิษฐ์\n ประว...,<PERSON> วงศ์เจริญ\nณ็ชพล</PERSON>\n วิศวกรปัญ...,วิศวกรปัญญาประดิษฐ์\n ประวัติการศึกษา\nมหาวิทย...
8,resume 3.pdf,re sume\n ประวัติส่วนตัว\n090-123-4567\npimcha...,re sume\n ประวัติส่วนตัว<PHONE>\n09</PHONE><PH...,ผู้ เชี่ยวชาญด้าน ai ที่มีประสบการณ์ 4 เขี\nใน...
9,resume 4.pdf,อาทิตย์\nรัตนวิจิตร\n วิศวกรปัญญาประดิษฐ์\nปร...,<PERSON> </PERSON><PERSON>อาทิตย์\nรัตนวิจิตร<...,วิศวกรปัญญาประดิษฐ์\nประสบการณํ ทำงาน\nวิศวกร ...


In [ ]:
!pip install langchain langchain_huggingface langchain_community

In [32]:
import os
from langchain import PromptTemplate, LLMChain, HuggingFaceHub
from huggingface_hub import InferenceClient
import concurrent.futures
import re
import pandas as pd
import requests
from langchain import PromptTemplate
from threading import Semaphore

## **DEFINE FUNCTION FOR PROMPT**

In [33]:
import re

def extract_relevant_text(text):
    pattern_prompt = r"PROMPT GENERATED:\s*(.*)"
    match_prompt = re.search(pattern_prompt, text, re.DOTALL)

    if match_prompt:
        extracted_text = match_prompt.group(1).strip()
    else:
        extracted_text = text.strip()

    keywords = ["###", "- **", "Human:", "Assistant:", "System:", "PROMPT", "---"]
    pattern_relevant = r'(' + '|'.join(re.escape(keyword) for keyword in keywords) + r')'
    match_relevant = re.search(pattern_relevant, extracted_text)

    if match_relevant:
        return extracted_text[:match_relevant.start()].strip()
    else:
        return extracted_text.strip()

### **BASE PROMPT (EN)**

In [50]:
baseprompt = """
YOU ARE A MASTER AI PROMPT ENGINEER, SPECIALIZING IN CRAFTING HIGHLY EFFECTIVE, TARGETED PROMPTS FOR MAXIMUM PERFORMANCE ACROSS VARIOUS DISCIPLINES AND TASKS.
YOUR OBJECTIVE IS TO DEVELOP A LASER-FOCUSED PROMPT THAT GUIDES A LANGUAGE MODEL TO EXECUTE THE USER'S INSTRUCTION WITH EXCEPTIONAL ACCURACY AND EFFICIENCY.

### TASK DESCRIPTION ###
- **USER INSTRUCTION**: {instruction}

### PROMPT DESIGN PRINCIPLES ###
1. **TASK IDENTIFICATION**: Accurately determine the nature of the task based on the user's instruction. This could include translation, problem-solving, creative writing, code generation, data analysis, etc.
   - For **translation** tasks, ensure the prompt instructs the model to maintain the original tone, context, and meaning while translating.
   - For **problem-solving** tasks, the prompt should guide the model to provide logical, step-by-step solutions, offering hints rather than direct answers.
   - For **code-related tasks**, direct the model to write, debug, or optimize code according to the specific criteria provided.
   - For **creative tasks**, like storytelling or idea generation, craft the prompt to inspire creativity while adhering to the provided context.
2. **LANGUAGE ALIGNMENT**: The prompt should mirror the language of the user's instruction. If multiple languages are involved, ensure the prompt maintains the required linguistic structure and accuracy.
3. **PRECISION AND CLARITY**: Ensure the prompt is concise and laser-focused on the task. Eliminate extraneous details and avoid unnecessary complexity. The prompt should be tailored precisely to the task’s specific requirements.
4. **CONTEXTUAL ADAPTATION**: Modify the prompt to fit the specific domain of the task. For example:
   - **Scientific Analysis**: Emphasize methodological accuracy and precision.
   - **Mathematics**: Prioritize logical flow and error-free calculations.
   - **Social Sciences**: Highlight critical analysis and context comprehension.
   - **Arts and Humanities**: Encourage creative interpretation within contextual boundaries.

### GUIDELINES FOR PROMPT CREATION ###
- AVOID PROVIDING DIRECT ANSWERS OR SOLUTIONS UNLESS EXPLICITLY REQUESTED.
- STAY STRICTLY WITHIN THE SCOPE OF THE USER'S INSTRUCTION AND LANGUAGE.
- REFRAIN FROM ADDING UNNECESSARY INFORMATION OR COMPLICATING THE PROMPT.
- AVOID CREATING PROMPTS THAT ARE TOO VAGUE, AMBIGUOUS, OR OVERLY COMPLEX FOR THE TASK.
- MAINTAIN FOCUS AND EFFICIENCY BY AVOIDING REPETITIVE OR REDUNDANT LANGUAGE.

### STRATEGIC THINKING PROCESS ###
1. Analyze the user’s instruction to accurately identify the task type and relevant domain.
2. Consider the linguistic and contextual requirements of the task.
3. Develop a prompt that is precise, clear, and optimized to achieve the task's objectives, tailored to the user's specific needs.
4. Review the prompt to ensure it adheres to all guidelines, maintaining consistency, relevance, and efficiency.

GENERATED PROMPT:
"""

In [54]:
import requests
from langchain import PromptTemplate

# Define the function to generate text based on the given prompt
def fnbase(instruction):
    formatted_prompt = baseprompt.replace("{instruction}", instruction)
    client = InferenceClient('https://ai-api.manageai.co.th/llm-model-02/')
    response = client.text_generation(formatted_prompt, **model_params)
    output = "".join(response)
    return output

def generate_prompt(instruction):
    result = fnbase(instruction)
    return result

# Define the model parameters
model_params = {
    "max_new_tokens": 1000,
    "temperature": 0.7,
    "top_p": 0.95,
    "repetition_penalty": 1.0
}

In [57]:
instruction = """
You are a CEO reviewing a resume for a potential candidate. Please evaluate the resume provided below, considering factors such as job qualifications, experience, skills, and overall presentation. Rate the resume on a scale from 1 to 10, where 1 is "very poor" and 10 is "outstanding". Provide a brief explanation for the score you assign.

Resume:

{resume}

Evaluation:

Score: [Your Score Here]
Strengths: [Highlight the strengths of the resume, such as relevant qualifications, strong experience, or excellent presentation.]
Areas for Improvement: [Identify areas where the resume could be improved, such as gaps in experience, unclear objectives, or formatting issues.]
Overall Impression: [Summarize your overall thoughts on the resume, including how well it fits the job qualifications and the potential impact on your decision.]
Score Range:

1-2: Very Poor – The resume lacks relevant qualifications, contains numerous errors, and does not meet basic job requirements.
3-4: Poor – The resume shows some relevant experience but has significant gaps or issues that impact its effectiveness.
5-6: Average – The resume is adequate and covers basic qualifications but lacks strong highlights or exceptional qualities.
7-8: Good – The resume is well-written, highlights relevant qualifications and experience effectively, with minor improvements needed.
9-10: Outstanding – The resume is exceptional, showcasing strong qualifications, experience, and skills in a highly professional manner.
"""
generated_prompt = generate_prompt(instruction)
extracted_text = extract_relevant_text(generated_prompt)
print(extracted_text)

Evaluate the following resume based on the criteria of job qualifications, experience, skills, and overall presentation. Assign a score from 1 to 10, with 1 being "very poor" and 10 being "outstanding". Provide a detailed analysis of the strengths and areas for improvement, and summarize your overall impression of the candidate's fit for the position. Consider the following score ranges:

- 1-2: Very Poor – The resume lacks relevant qualifications, contains numerous errors, and does not meet basic job requirements.
- 3-4: Poor – The resume shows some relevant experience but has significant gaps or issues that impact its effectiveness.
- 5-6: Average – The resume is adequate and covers basic qualifications but lacks strong highlights or exceptional qualities.
- 7-8: Good – The resume is well-written, highlights relevant qualifications and experience effectively, with minor improvements needed.
- 9-10: Outstanding – The resume is exceptional, showcasing strong qualifications, experience,

#### **RESUME PROMPT**

In [59]:
resume_prompt = """
Evaluate the following resume based on the criteria of job qualifications, experience, skills, and overall presentation. Assign a score from 1 to 10, with 1 being "very poor" and 10 being "outstanding". Provide a detailed analysis of the strengths and areas for improvement, and summarize your overall impression of the candidate's fit for the position. Consider the following score ranges:

- 1-2: Very Poor – The resume lacks relevant qualifications, contains numerous errors, and does not meet basic job requirements.
- 3-4: Poor – The resume shows some relevant experience but has significant gaps or issues that impact its effectiveness.
- 5-6: Average – The resume is adequate and covers basic qualifications but lacks strong highlights or exceptional qualities.
- 7-8: Good – The resume is well-written, highlights relevant qualifications and experience effectively, with minor improvements needed.
- 9-10: Outstanding – The resume is exceptional, showcasing strong qualifications, experience, and skills in a highly professional manner.

Resume:
{resume}

Evaluation:
Score: [Your Score Here]
Strengths: [Highlight the strengths of the resume, such as relevant qualifications, strong experience, or excellent presentation.]
Areas for Improvement: [Identify areas where the resume could be improved, such as gaps in experience, unclear objectives, or formatting issues.]
Overall Impression: [Summarize your overall thoughts on the resume, including how well it fits the job qualifications and the potential impact on your decision.]

Please note that you are evaluating the resume as if you were a CEO reviewing a candidate for a high-level position in your company. Consider the resume's alignment with the job requirements and your expectations for a successful candidate in this role.
"""


In [60]:
import requests
import re
from threading import Semaphore

# Semaphore to limit concurrent API requests
semaphore = Semaphore(50)  # Adjust the number as needed

# Define the model parameters
model_params = {
    "max_new_tokens": 512,
    "temperature": 0.5,
    "top_p": 0.95,
    "repetition_penalty": 1.0
}

def fn(resume):
    formatted_prompt = resume_prompt_th.replace("{resume}", resume)
    client = InferenceClient('https://ai-api.manageai.co.th/llm-model-02/')
    response = client.text_generation(formatted_prompt, **model_params)
    output = "".join(response)
    return output

# Define the function to generate answer
def generate_answer(resume):
    result = fn(resume)
    return result

In [61]:
print(df['Cleaned_Text'].iloc[0])

เป้าหมายในการทำงาน
นำความเชี่ยวชาญด้านการวิเคราะห์และการจัดการโครงการมาช่วยองค์กรปรับปรุงกระบวนการ
ทำงานและบรรลุเป้าหมายธุรกิจอย่างมีประสิทธิภาพ
ทักษะ
การวิเคราะห์เชิงกลยุทธ์ swot pestel
การจัดการโครงการ microsoft project asana
การสื่อสาร การนำเสนอการจัดทำรายงาน
การจัดการข้อมูล excel. sql power bi
การศึกษา
 ปริญญาโท สาขาวิทยาการจัดการ
2015
2017
มหาวิทยาลัยเกษตรศาสตร์
ประสบการณ์การทำงาน
senior business analyst บริษัท strategic partners
ทำงานร่วมกับทีมผู้บริหารเพื่อระบุความต้องการและโอกาสในการปรับปรุงกระบวนการธุรกิจ
พัฒนา roadmap สำหรับการเปลี่ยนแปลงและปรับปรุงระบบเทคโนโลยีสารสนเทศ
 วิเคราะห์และนำเสนอข้อมูลสำคัญที่มีผลต่อการตัดสินใจเชิงกลยุทธ์
business analyst บริษัท delta innovations


In [62]:
resume = """
เป้าหมายในการทำงาน
นำความเชี่ยวชาญด้านการวิเคราะห์และการจัดการโครงการมาช่วยองค์กรปรับปรุงกระบวนการ
ทำงานและบรรลุเป้าหมายธุรกิจอย่างมีประสิทธิภาพ
ทักษะ
การวิเคราะห์เชิงกลยุทธ์ swot pestel
การจัดการโครงการ microsoft project asana
การสื่อสาร การนำเสนอการจัดทำรายงาน
การจัดการข้อมูล excel. sql power bi
การศึกษา
 ปริญญาโท สาขาวิทยาการจัดการ
2015
2017
มหาวิทยาลัยเกษตรศาสตร์
ประสบการณ์การทำงาน
senior business analyst บริษัท strategic partners
ทำงานร่วมกับทีมผู้บริหารเพื่อระบุความต้องการและโอกาสในการปรับปรุงกระบวนการธุรกิจ
พัฒนา roadmap สำหรับการเปลี่ยนแปลงและปรับปรุงระบบเทคโนโลยีสารสนเทศ
 วิเคราะห์และนำเสนอข้อมูลสำคัญที่มีผลต่อการตัดสินใจเชิงกลยุทธ์
business analyst บริษัท delta innovations
"""
generated_answer = generate_answer(resume)
print(generated_answer)

{resume}
Score: 8

Strengths:
The resume is well-organized and presents the candidate's qualifications and experience in a clear and concise manner. The candidate has a strong educational background, having earned a Bachelor of Science in Computer Science from a reputable university. They have over 10 years of experience in software development, with a focus on Java and Python programming languages. The candidate has worked on multiple projects, including developing a web-based application for a major e-commerce company and designing a data processing system for a financial institution. They have also led a team of developers and have experience with agile development methodologies. The candidate has received several awards for their work, including the "Outstanding Employee of the Year" award from their previous employer. They have also completed several relevant certifications, including the Certified Scrum Master (CSM) and the Oracle Certified Professional (OCP) in Java.

Areas for 

### **BASE PROMPT (TH)**

In [92]:
baseprompt = """
YOU ARE A MASTER AI PROMPT ENGINEER, SPECIALIZING IN CRAFTING HIGHLY EFFECTIVE, TARGETED PROMPTS FOR MAXIMUM PERFORMANCE ACROSS VARIOUS DISCIPLINES AND TASKS.
YOUR OBJECTIVE IS TO DEVELOP A LASER-FOCUSED PROMPT THAT GUIDES A LANGUAGE MODEL TO EXECUTE THE USER'S INSTRUCTION WITH EXCEPTIONAL ACCURACY AND EFFICIENCY.

### TASK DESCRIPTION ###
- **USER INSTRUCTION**: {instruction}

### PROMPT DESIGN PRINCIPLES ###
1. **TASK IDENTIFICATION**: Accurately determine the nature of the task based on the user's instruction. This could include translation, problem-solving, creative writing, code generation, data analysis, etc.
   - For **translation** tasks, ensure the prompt instructs the model to maintain the original tone, context, and meaning while translating.
   - For **problem-solving** tasks, the prompt should guide the model to provide logical, step-by-step solutions, offering hints rather than direct answers.
   - For **code-related tasks**, direct the model to write, debug, or optimize code according to the specific criteria provided.
   - For **creative tasks**, like storytelling or idea generation, craft the prompt to inspire creativity while adhering to the provided context.
2. **LANGUAGE ALIGNMENT**: The prompt should mirror the language of the user's instruction. If multiple languages are involved, ensure the prompt maintains the required linguistic structure and accuracy.
3. **PRECISION AND CLARITY**: Ensure the prompt is concise and laser-focused on the task. Eliminate extraneous details and avoid unnecessary complexity. The prompt should be tailored precisely to the task’s specific requirements.
4. **CONTEXTUAL ADAPTATION**: Modify the prompt to fit the specific domain of the task. For example:
   - **Scientific Analysis**: Emphasize methodological accuracy and precision.
   - **Mathematics**: Prioritize logical flow and error-free calculations.
   - **Social Sciences**: Highlight critical analysis and context comprehension.
   - **Arts and Humanities**: Encourage creative interpretation within contextual boundaries.

### GUIDELINES FOR PROMPT CREATION ###
- AVOID PROVIDING DIRECT ANSWERS OR SOLUTIONS UNLESS EXPLICITLY REQUESTED.
- STAY STRICTLY WITHIN THE SCOPE OF THE USER'S INSTRUCTION AND LANGUAGE.
- REFRAIN FROM ADDING UNNECESSARY INFORMATION OR COMPLICATING THE PROMPT.
- AVOID CREATING PROMPTS THAT ARE TOO VAGUE, AMBIGUOUS, OR OVERLY COMPLEX FOR THE TASK.
- MAINTAIN FOCUS AND EFFICIENCY BY AVOIDING REPETITIVE OR REDUNDANT LANGUAGE.

### STRATEGIC THINKING PROCESS ###
1. Analyze the user’s instruction to accurately identify the task type and relevant domain.
2. Consider the linguistic and contextual requirements of the task.
3. Develop a prompt that is precise, clear, and optimized to achieve the task's objectives, tailored to the user's specific needs.
4. Review the prompt to ensure it adheres to all guidelines, maintaining consistency, relevance, and efficiency.

GENERATED PROMPT:
"""

In [93]:
import requests
from langchain import PromptTemplate

# Define the function to generate text based on the given prompt
def fnbase(instruction):
    formatted_prompt = baseprompt.replace("{instruction}", instruction)
    client = InferenceClient('https://ai-api.manageai.co.th/llm-model-02/')
    response = client.text_generation(formatted_prompt, **model_params)
    output = "".join(response)
    return output

def generate_prompt(instruction):
    result = fnbase(instruction)
    return result

# Define the model parameters
model_params = {
    "max_new_tokens": 1000,
    "temperature": 0.7,
    "top_p": 0.95,
    "repetition_penalty": 1.0
}

In [101]:
instruction = """
คุณเป็น CEO ที่กำลังทบทวนเรซูเม่ของผู้สมัครงาน โปรดประเมินเรซูเม่ที่ให้ไว้ด้านล่าง โดยพิจารณาปัจจัยต่างๆ เช่น คุณสมบัติสำหรับงาน ประสบการณ์ ทักษะ และการนำเสนอโดยรวม
ให้คะแนนเรซูเม่ในระดับ 1 ถึง 10 โดยที่ 1 หมายถึง "แย่มาก" และ 10 หมายถึง "ยอดเยี่ยม" โปรดอธิบายสั้นๆ เกี่ยวกับคะแนนที่คุณให้ และพิจารณาว่าควรเรียกสัมภาษณ์ผู้สมัครงานนี้สำหรับตำแหน่งที่เขาส่งมา หรือไม่ เป็นภาษาไทย

เรซูเม่:

{resume}

การประเมิน:

คะแนน: [คะแนนของคุณที่นี่]
จุดแข็ง: [ระบุจุดแข็งของเรซูเม่ เช่น คุณสมบัติที่เกี่ยวข้อง ประสบการณ์ที่แข็งแกร่ง หรือการนำเสนอที่ยอดเยี่ยม]
จุดที่ควรปรับปรุง: [ระบุจุดที่เรซูเม่สามารถปรับปรุงได้ เช่น ช่องว่างในประสบการณ์ วัตถุประสงค์ที่ไม่ชัดเจน หรือปัญหาด้านการจัดรูปแบบ]
ความประทับใจโดยรวม: [สรุปความคิดของคุณโดยรวมเกี่ยวกับเรซูเม่ รวมถึงความเหมาะสมกับคุณสมบัติของงานและผลกระทบต่อการตัดสินใจของคุณ]
ระดับคะแนน:

1-2: แย่มาก – เรซูเม่ขาดคุณสมบัติที่เกี่ยวข้อง มีข้อผิดพลาดมากมาย และไม่ตรงตามข้อกำหนดพื้นฐานของงาน
3-4: แย่ – เรซูเม่แสดงถึงประสบการณ์ที่เกี่ยวข้องบางอย่าง แต่มีช่องว่างหรือปัญหาสำคัญที่ส่งผลต่อประสิทธิภาพ
5-6: ปานกลาง – เรซูเม่ครอบคลุมคุณสมบัติพื้นฐานได้ดีพอสมควร แต่ขาดความโดดเด่นหรือคุณสมบัติพิเศษ
7-8: ดี – เรซูเม่เขียนได้ดี แสดงถึงคุณสมบัติและประสบการณ์ที่เกี่ยวข้องได้อย่างมีประสิทธิภาพ โดยต้องการการปรับปรุงเล็กน้อย
9-10: ยอดเยี่ยม – เรซูเม่นำเสนอคุณสมบัติ ประสบการณ์ และทักษะอย่างโดดเด่นและเป็นมืออาชีพ
"""
generated_prompt = generate_prompt(instruction)
extracted_text = extract_relevant_text(generated_prompt)
print(extracted_text)

คุณเป็น CEO ที่กำลังทบทวนเรซูเม่ของผู้สมัครงาน. โปรดประเมินเรซูเม่ด้านล่างโดยพิจารณาคุณสมบัติ, ประสบการณ์, ทักษะ, และการนำเสนอโดยรวมของผู้สมัคร. ให้คะแนนเรซูเม่ในระดับ 1 ถึง 10 โดยที่ 1 หมายถึง "แย่มาก" และ 10 หมายถึง "ยอดเยี่ยม". ให้อธิบายสั้นๆ เกี่ยวกับคะแนนที่คุณให้, จุดแข็งและจุดที่ควรปรับปรุงของเรซูเม่, และความประทับใจโดยรวม. พิจารณาว่าควรเรียกสัมภาษณ์ผู้สมัครงานนี้หรือไม่ สำหรับตำแหน่งที่เขาส่งมา. ภาษาที่ใช้ในการตอบต้องเป็นภาษาไทย.

เรซูเม่:

{resume}

การประเมิน:

คะแนน: [คะแนนของคุณที่นี่]
จุดแข็ง: [ระบุจุดแข็งของเรซูเม่]
จุดที่ควรปรับปรุง: [ระบุจุดที่เรซูเม่สามารถปรับปรุงได้]
ความประทับใจโดยรวม: [สรุปความคิดของคุณโดยรวมเกี่ยวกับเรซูเม่]

ระดับคะแนน:

1-2: แย่มาก
3-4: แย่
5-6: ปานกลาง
7-8: ดี
9-10: ยอดเยี่ยม

โปรดประเมินเรซูเม่ตามระดับคะแนนที่กำหนดไว้ด้านบน, ให้คำอธิบายที่ชัดเจนเกี่ยวกับคะแนนที่คุณให้ และวิเคราะห์ว่าเรซูเม่ของผู้สมัครนี้เหมาะสมกับคุณสมบัติของงานที่กำหนดไว้หรือไม่. หากมีความเห็นเพิ่มเติมเกี่ยวกับการสัมภาษณ์ผู้สมัครนี้, โปรดระบุในส่วนความประทับใจโดยรวม. คุณควรพิจ

#### **RESUME PROMPT**

In [102]:
resume_prompt_th = """
คุณเป็น CEO ที่กำลังทบทวนเรซูเม่ของผู้สมัครงาน. โปรดประเมินเรซูเม่ด้านล่างโดยพิจารณาคุณสมบัติ, ประสบการณ์, ทักษะ, และการนำเสนอโดยรวมของผู้สมัคร.
ให้คะแนนเรซูเม่ในระดับ 1 ถึง 10 โดยที่ 1 หมายถึง "แย่มาก" และ 10 หมายถึง "ยอดเยี่ยม". ให้อธิบายสั้นๆ เกี่ยวกับคะแนนที่คุณให้, จุดแข็งและจุดที่ควรปรับปรุงของเรซูเม่, และความประทับใจโดยรวม.
พิจารณาว่าควรเรียกสัมภาษณ์ผู้สมัครงานนี้หรือไม่ สำหรับตำแหน่งที่เขาส่งมา. ภาษาที่ใช้ในการตอบต้องเป็นภาษาไทย.

เรซูเม่:

{resume}

การประเมิน:

คะแนน: [คะแนนของคุณที่นี่]
จุดแข็ง: [ระบุจุดแข็งของเรซูเม่]
จุดที่ควรปรับปรุง: [ระบุจุดที่เรซูเม่สามารถปรับปรุงได้]
ความประทับใจโดยรวม: [สรุปความคิดของคุณโดยรวมเกี่ยวกับเรซูเม่]

ระดับคะแนน:

1-2: แย่มาก
3-4: แย่
5-6: ปานกลาง
7-8: ดี
9-10: ยอดเยี่ยม

โปรดประเมินเรซูเม่ตามระดับคะแนนที่กำหนดไว้ด้านบน, ให้คำอธิบายที่ชัดเจนเกี่ยวกับคะแนนที่คุณให้ และวิเคราะห์ว่าเรซูเม่ของผู้สมัครนี้เหมาะสมกับคุณสมบัติของงานที่กำหนดไว้หรือไม่. หากมีความเห็นเพิ่มเติมเกี่ยวกับการสัมภาษณ์ผู้สมัครนี้, โปรดระบุในส่วนความประทับใจโดยรวม. คุณควรพิจารณาทั้งคุณสมบัติที่เกี่ยวข้อง, ประสบการณ์, และทักษะที่จำเป็นสำหรับตำแหน่งที่ผู้สมัครสมัครงาน.

"""


In [103]:
import requests
import re
from threading import Semaphore

# Semaphore to limit concurrent API requests
semaphore = Semaphore(50)  # Adjust the number as needed

# Define the model parameters
model_params = {
    "max_new_tokens": 512,
    "temperature": 0.5,
    "top_p": 0.95,
    "repetition_penalty": 1.0
}

def fn(resume):
    formatted_prompt = resume_prompt_th.replace("{resume}", resume)
    client = InferenceClient('https://ai-api.manageai.co.th/llm-model-02/')
    response = client.text_generation(formatted_prompt, **model_params)
    output = "".join(response)
    return output

# Define the function to generate answer
def generate_answer(resume):
    result = fn(resume)
    return result

In [104]:
resume = """
ux/ ui designer
in

การศึกษา
ฉันเป็นคนที่ชอบเรียนรู้และมี
studio shodwe university
2023
ความกระตือรือรันในการทำงาน
ฉันเว่าการเรียนรู้ไม่มีที่สิ้นสุด
คณะสถาปัตยกรรมศาสตร์และการ
และทุกวันคือโอกาสที่จะพัฒนา
 ออกแบบ ภาควิชานิเทศศิลป์
 เกรดเฉลีย 3.7
ตนเอง เราอาจพบกันในห้องเรียน
หรือในการทำงานเป็นทีม ทั่้งนี้
เพื่อแลกเปลี่ยนความรู้และ
ประสบการณ์การทำงาน
 ประสบการณ์ที่มีคุณค่า
2023
"""
generated_answer = generate_answer(resume)
print(generated_answer)

การประเมิน:

คะแนน: 4
จุดแข็ง: เรซูเม่มีการระบุตำแหน่งที่สมัครและมีการระบุการศึกษาที่เกี่ยวข้อง ซึ่งคือ คณะสถาปัตยกรรมศาสตร์และการออกแบบ ภาควิชานิเทศศิลป์ ที่มหาวิทยาลัยชื่อ Studio Shodwe University และมีเกรดเฉลี่ย 3.7 ซึ่งถือว่าเป็นเกรดที่ดี
จุดที่ควรปรับปรุง: ข้อมูลที่เกี่ยวกับประสบการณ์การทำงาน ทักษะที่มี และผลงานที่สร้างสรรค์ยังขาดหาย ซึ่งเป็นข้อมูลที่สำคัญสำหรับตำแหน่ง UX/UI Designer นอกจากนี้ การนำเสนอเรซูเม่ยังขาดความชัดเจน ดังนั้นควรปรับปรุงเรื่องการจัดรูปแบบและการใช้ภาษา
ความประทับใจโดยรวม: เรซูเม่ยังขาดความสมบูรณ์และไม่ชัดเจนพอ แต่ยังมีความหวังที่จะพัฒนาได้ เนื่องจากมีเกรดที่ดีและมีการศึกษาที่เกี่ยวข้องกับตำแหน่งที่สมัคร
ควรเรียกสัมภาษณ์ผู้สมัครนี้หรือไม่: ควรขอให้ผู้สมัครส่งเรซูเม่ที่มีรายละเอียดมากขึ้นหรือส่งพอร์ตโฟลิโอเพื่อประเมินความสามารถที่แท้จริงของเขา. ถ้าได้รับข้อมูลเพิ่มเติมที่ดี อาจควรเรียกสัมภาษณ์. 

ระดับคะแนน: 3-4: แย่

ขอบคุณที่ให้ความสนใจในตำแหน่งนี้. หากคุณสามารถปรับปรุงเรซูเม่ของคุณให้สมบูรณ์มากขึ้น คุณอาจม


In [105]:
resume = """
ขอสมัครตำแหน่ง AI Engineer ฉันเก่งที่สุด โปรดรับฉันเข้าทำงาน
"""
generated_answer = generate_answer(resume)
print(generated_answer)

การประเมิน:

คะแนน: 2
จุดแข็ง: ไม่มี
จุดที่ควรปรับปรุง: ผู้สมัครควรให้รายละเอียดเกี่ยวกับประสบการณ์การทำงานที่เกี่ยวข้อง, คุณสมบัติทางวิชาการ, ทักษะเฉพาะทาง, และผลงานที่เกี่ยวข้อง. การอ้างอิงถึงความสามารถที่ดีที่สุดไม่เพียงพอในการแสดงให้เห็นว่าคุณสมบัติและทักษะของคุณเหมาะสมกับตำแหน่งที่สมัคร.
ความประทับใจโดยรวม: ความประทับใจโดยรวมคือผู้สมัครยังขาดการเตรียมตัวในการสมัครงาน. การแสดงความมั่นใจเป็นสิ่งที่ดี แต่ยังต้องมีการสนับสนุนด้วยข้อมูลที่เป็นรูปธรรม. 

ตามที่ประเมิน, ฉันไม่แนะนำให้เรียกผู้สมัครนี้เข้าสัมภาษณ์สำหรับตำแหน่ง AI Engineer ในขณะนี้. แต่ถ้าผู้สมัครสามารถปรับปรุงเรซูเม่ของตนให้ละเอียดมากขึ้น, ควรทบทวนการสมัครของเขานั้นอีกครั้ง. ขอแนะนำให้ผู้สมัครเรียนรู้เพิ่มเติมเกี่ยวกับการสร้างเรซูเม่ที่มีประสิทธิภาพ. ความมั่นใจเป็นสิ่งที่ดี แต่ต้องมีการสนับสนุนด้วยข้อมูลที่ชัดเจนและเป็นรูปธรรม.


## **COMPARE**
`RAW_TEXT` vs `CLEANED_TEXT` ให้คะแนนเท่ากัน

> 1

In [111]:
resume = """
เป้าหมายในการ
 มุ่งมันในการนำทักษะด้านการจัดการและการวิเคราะห์จากประสบการณ์
การทำงานในสายงานการจัดการโครงการมาใช้ในการวิเคราะห์และ
ทำงาน
เพือเพิมประสิทธิภาพและความสามารถใน
 ปรับปรุงกระบวนการธุรกิจ
การแข่งขันขององค์กร
 ประสบการณ์การทำงาน
 บริษัท abc construction


 project manager
แลและจัดการโครงการก่อสร้างขนาดใหญ่ ตั้งแต่เริมต้นจนจบโครงการ
ดู
ประสานงานระหว่างทีมงานต่างๆ เพือให้โครงการสำเร็จตามแผนทีกำหนด
 วิเคราะห์และจัดการความเสียงของโครงการ พร้อมทังปรับแผนการดำเนินงานให้เหมาะสม
 บริษัท ixyz logistics

assistant manager
 ควบคุมและดูแลการปฏิบัติงานประจำวันของแผนกโลจิสติกส์
 วิเคราะห์ ข้อมูลการจัดส่งและจัดทำรายงานเพือปรับปรุงประสิทธิภาพ
ประสานงานกับลูกค้าและชัพพลายเออร์เพื่อแก้ไขปัญหาที่เกิดขึ้น
การศึกษา
 ปริญญาตรี สาขาวิศวกรรมโยธา
มหาวิทยาลัยเทคโนโลยีพระจอมเกล้าธนบุรี
"""
generated_answer = generate_answer(resume)
print(generated_answer)

Assistant: คะแนน: 8
จุดแข็ง: เรซูเม่นี้มีการนำเสนอที่ชัดเจนและมีโครงสร้างที่ดี ผู้สมัครมีประสบการณ์ในการจัดการโครงการที่มีความซับซ้อนและสามารถทำงานได้ดีในสภาพแวดล้อมที่ต้องมีการประสานงานกับทีมงานหลากหลาย ทักษะการวิเคราะห์และแก้ปัญหาที่ดี ผ่านการศึกษาทางวิศวกรรมโยธา ซึ่งเป็นหลักสูตรที่มีความยากและต้องการทักษะทางคณิตศาสตร์และความสามารถในการวิเคราะห์ที่สูง
จุดที่ควรปรับปรุง: ควรเพิ่มรายละเอียดเกี่ยวกับการศึกษาของคุณ เช่น GPA หรือเกียรตินิยมที่ได้รับ และควรเพิ่มรายละเอียดเกี่ยวกับทักษะเฉพาะทางที่คุณมี เช่น ทักษะทางด้านซอฟต์แวร์หรือโปรแกรมที่คุณมีความชำนาญ
ความประทับใจโดยรวม: เรซูเม่ของผู้สมัครนี้น่าสนใจและมีคุณสมบัติที่น่าสนใจ ควรเรียกผู้สมัครมาสัมภาษณ์เพื่อสำรวจความรู้และความสามารถของพวกเขาในรายละเอียดเพิ่มเติม. ความรู้และประสบการณ์ที่ผู้สมัครมีนั้นสามารถนำไปปรับใช้ในตำแหน่งที่สมัครได้. แต่ควรให้ผู้สมัครเตรียมข้อมูลเพิ่มเติมเกี่ยวกับทักษะเฉพาะทางและประสบการณ์ที่เกี่ยวข้อง.


## **BATCH**

In [126]:
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

# Define the processing function
def generate_prompt_answer_optimized(row):
    resume = row.get('Raw_Text', "")
    result = generate_answer(resume)
    return result

# Define the batch processing function
def process_batch(batch_df):
    return [generate_prompt_answer_optimized(row) for _, row in batch_df.iterrows()]

# Batch processing with threading
def batch_process(df, batch_size=32):
    results = [None] * len(df)  # Initialize a list to store results in original order
    with ThreadPoolExecutor() as executor:
        futures = {}
        for i in range(0, len(df), batch_size):
            batch_df = df.iloc[i:i + batch_size]
            future = executor.submit(process_batch, batch_df)
            futures[future] = (i, i + batch_size)  # Store the index range for each future

        for future in as_completed(futures):
            start_idx, end_idx = futures[future]
            batch_results = future.result()
            results[start_idx:end_idx] = batch_results

    return results

> 6 mins

In [118]:
df['Evaluation'] = batch_process(df, batch_size=32)

In [127]:
df['Evaluation_Raw_text'] = batch_process(df, batch_size=32)

In [120]:
import re

def extract_score(text):
    match = re.search(r'คะแนน:\s*([0-9]+(?:\.[0-9]+)?)', text)
    if match:
        return float(match.group(1))
    return text

In [ ]:
df['Score'] = df['Evaluation_Raw_text'].apply(extract_score)

In [121]:
df['Score'] = df['Evaluation'].apply(extract_score)

In [122]:
df

,PDF File,Raw_Text,Tagged_Text,Cleaned_Text,Evaluation,Score
0,Resume BA 2.pdf,พิมพ์นารา วัฒนชัย\nที่อยู่:\n789 ถนนรัชดาภิเษก...,<PERSON>พ</PERSON><PERSON>พิมพ์</PERSON><PERSO...,เป้าหมายในการทำงาน\nนำความเชี่ยวชาญด้านการวิเค...,คะแนน: 8\nจุดแข็ง: ผู้สมัครมีประสบการณ์ที่เกี่...,8.0
1,Resume BA 3.pdf,จิตรลดา\n086-234-5678\n jitlada.s@domain.com\...,<PERSON> </PERSON><PERSON>จิตร</PERSON><PERSON...,เสริม\n business analyst\nเป้าหมายในการทำงาน\n...,การประเมิน:\n\nคะแนน: 8\nจุดแข็ง: เรซูเม่ของผู...,8.0
2,Resume ba 5.pdf,081-456-7890\nนัทธมน ฺรัตนโชติ\n nattamon. rad...,<PHONE>08</PHONE><PERSON>1-4</PERSON><PERSON>5...,เป้าหมายในการ\n มุ่งมันในการนำทักษะด้านการจัดก...,"ด้วยความเคารพ,\n[ชื่อของคุณ]\nคะแนน: 8\nจุดแข็...",8.0
3,Resume ba2.pdf,ศุภกิจฺ มงคลชัย\n089-234-5678\n supakit.m@dom...,<PERSON> </PERSON><PERSON>ศุภ</PERSON><PERSON>...,ประสบการณ์การ\nเป้าหมายในการทำงาน\nfinancial a...,คะแนน: 8\nจุดแข็ง: ผู้สมัครมีประสบการณ์ที่เกี่...,8.0
4,Resume baa.pdf,"ธนกร อินทรีย์พงษ์\n ที่อยู่:\n456 ถนนพระราม 3,...",<PERSON>ธ</PERSON><PERSON>ธน</PERSON><PERSON>ก...,เป้าหมายในการทำงาน\nมุ่งมันที่จะใช้ความรู้และท...,ระดับคะแนน: 8\n\nจุดแข็ง: \nผู้สมัครมีประสบการ...,8.0
5,resume 6.pdf,linkedin:\nนภัสสร วิวัฒนาวงศ์\nlinkedin.com i...,linkedin:<PERSON>\n</PERSON><PERSON>น</PERSON>...,in\n\n วิศวกรปัญญาประดิษฐ์\n\n ประสบการณ์การทำ...,การประเมิน:\n\nคะแนน: 8\nจุดแข็ง: ผู้สมัครมีปร...,8.0
6,resume 7.pdf,ปัญญา วิริยะชัย\n วิศวกรปัญญาประดิษฐ์\nประสบกา...,<PERSON>ป</PERSON><PERSON>ปัญญา วิริยะชัย</PER...,วิศวกรปัญญาประดิษฐ์\nประสบการณ์การฝึกงาน\nบัณฑ...,คะแนน: 8\nจุดแข็ง: เรซูเม่ของผู้สมัครนี้มีความ...,8.0
7,resume 2.pdf,วงศ์เจริญ\nณ็ชพล\n วิศวกรปัญญาประดิษฐ์\n ประว...,<PERSON> วงศ์เจริญ\nณ็ชพล</PERSON>\n วิศวกรปัญ...,วิศวกรปัญญาประดิษฐ์\n ประวัติการศึกษา\nมหาวิทย...,การประเมิน:\n\nคะแนน: 8.5\nจุดแข็ง: เรซูเม่ของ...,8.5
8,resume 3.pdf,re sume\n ประวัติส่วนตัว\n090-123-4567\npimcha...,re sume\n ประวัติส่วนตัว<PHONE>\n09</PHONE><PH...,ผู้ เชี่ยวชาญด้าน ai ที่มีประสบการณ์ 4 เขี\nใน...,การประเมิน:\n\nคะแนน: 6\nจุดแข็ง: ผู้สมัครมีปร...,6.0
9,resume 4.pdf,อาทิตย์\nรัตนวิจิตร\n วิศวกรปัญญาประดิษฐ์\nปร...,<PERSON> </PERSON><PERSON>อาทิตย์\nรัตนวิจิตร<...,วิศวกรปัญญาประดิษฐ์\nประสบการณํ ทำงาน\nวิศวกร ...,ระดับคะแนน: 8\n\nจุดแข็ง:\n- ผู้สมัครมีประสบกา...,8.0


## **JOB QUALIFIED**

In [136]:
resume_prompt_en = """
You are a highly experienced CEO with a reputation for demanding excellence. Your company only hires the best of the best, and you approach each resume with a critical eye, carefully weighing the value of each candidate. As you review the resume below for a Senior AI Engineer position within the Technical Labs team at SCB 10X, consider whether this candidate truly stands out in a competitive field.

**Job Summary:**
The Senior AI Engineer position will be part of the Technical Labs team of SCB 10X. Some responsibilities include but are not limited to:

**Job Descriptions:**
- Lead AI activities and development for the lab team.
- Support the Venture Capital team to evaluate prospective companies, particularly on their AI technology capability and feasibility.
- Stay up to date on the latest technology developments relevant to the business domain.
- Conduct research and development processes for AI strategies.
- Collaborate with cross-functional teams to explore ideas where AI could be developed into any promising business.
- Provide technical support in product idea validation.
- Work with industry experts, other engineers, and scientists to develop proof of concept for validated ideas.
- Contribute to AI communities through activities such as knowledge sharing, technical blog writing, etc.

**Qualifications:**
- Experience in formulating and strategizing AI solutions.
- Hands-on knowledge in AI, machine learning, deep learning, PyTorch, Python, NLP.
- Experience in one or more specialized machine learning areas, e.g., NLP, deep learning, recommendation systems, computer vision, conversational AI, and reinforcement learning.
- Proven experience in applying AI to practical and comprehensive technology solutions.
- Knowledge of various deep learning model architectures.
- Strong ability to modify deep learning model architectures.
- Experience with training deep learning models on large datasets.
- Strong programming skills in Python and proficiency with relevant libraries such as Numpy, Pandas, and Matplotlib.
- Strong team player: prioritize work and organize your time to support others efficiently.
- Excellent analytical thinking and problem-solving skills.
- Excellent written and verbal communication skills.
- Self-starter and fast learner in working with new areas/topics.
- Solid computer science fundamentals in statistical learning theory, algorithms, and data structures are preferred.
- Deep knowledge of one or more industrial domains is a plus.

As you evaluate this resume, keep in mind the high standards your company maintains. Does this candidate demonstrate the exceptional qualifications, experience, and skills necessary to excel in this role? Consider whether their achievements truly differentiate them in a competitive market, and whether their background reflects a deep understanding of AI that aligns with the rigorous demands of this position.

Resume:

{resume}

Evaluation:

Score: [Your score here]
Strengths: [Identify the strengths of the resume]
Areas for Improvement: [Identify areas where the resume could be improved]
Overall Impression: [Summarize your overall thoughts on the resume]

**Scoring Scale:**

1-2: Very Poor - This candidate shows significant gaps in qualifications, experience, or skills and is unlikely to succeed in this role.
3-4: Poor - The candidate has some relevant experience or skills but does not meet the high standards expected for this role.
5-6: Average - The candidate meets the basic requirements but does not stand out among their peers.
7-8: Good - The candidate has strong qualifications and experience but may lack the exceptional qualities needed to excel.
9-10: Excellent - The candidate is outstanding in every aspect and clearly exceeds the expectations for this role.

Be strict in your evaluation and consider the real-world impact of this candidate's potential contribution. Provide a clear, well-reasoned explanation for the score you assign, and analyze whether this resume justifies moving the candidate forward in the hiring process. Reflect on whether this person is truly worth investing in for an interview, given the competitive nature of the industry and the high standards of your company.
"""


In [137]:
import requests
import re
from threading import Semaphore

# Semaphore to limit concurrent API requests
semaphore = Semaphore(50)  # Adjust the number as needed

# Define the model parameters
model_params = {
    "max_new_tokens": 1500,
    "temperature": 0.5,
    "top_p": 0.95,
    "repetition_penalty": 1.0
}

def fn(resume):
    formatted_prompt = resume_prompt_en.replace("{resume}", resume)
    client = InferenceClient('https://ai-api.manageai.co.th/llm-model-02/')
    response = client.text_generation(formatted_prompt, **model_params)
    output = "".join(response)
    return output

# Define the function to generate answer
def generate_answer(resume):
    result = fn(resume)
    return result

In [138]:
resume = """
วิศวกรปัญญาประดิษฐ์
 ประสบการณ์ทำงาน

 วิศวกร ai ที่มีประสบการณ์ 5 ปีในการพัฒนา
 วิศวกร ai อาวุโส
และการเรียนรู้ของเครื่อง เชี่ยวชาญใน
ระบบ ai
 บริษัท สตาร์ทอัพ เอไอ

 เทคนิค reinforcement learning และการ

ปัจจุบัน
ประมวลผลภาษาธรรมชาติ (nlp) ใช้ python

และ pytorch ในการพัฒนาโมเดล ai สำหรับ
พัฒนาโมเดล reinforcement learning เพื่อใช้ไนระบบการ
งานด้านการวิเคราะห์ข้อมูลและการประมวลผล
ปรับตัวอัตโนมัติในโปรเจกต์ถ้านการขนส่งและการโลจิสติกส์
 อัตโนมัติ
ออกแบบและฝึกอบรมโมเดล nlp สำหรับระบบการจัดการคำถาม
ที่ตอบอัตโนมัติrintiexa
นักวิจัย ai
สถาบันวิจัยเทคโนโลยีสารสนเทศแห่งประเทศไทยr


nakit
กำการวิจัยและพัฒนาเทคโนโลยีการเรียนรู้ของเครื่องสำหรับการ
ประมวลผลข้อมูลขนาดใหญ่
ภาษา
ภาษาไทย
การศึกษา
ภาษาอังกฤษ
มหาวิทยาลัยเทคโนโลยีพระจอมเกล้าธนบุรี
"""
generated_answer = generate_answer(resume)
print(generated_answer)

Evaluation:

Score: 7

Strengths: 
- The candidate has 5 years of experience in AI engineering, which indicates a solid foundation in the field.
- They have expertise in reinforcement learning and NLP, which are key areas for the role, and have practical experience using Python and PyTorch, demonstrating hands-on knowledge.
- They have worked on projects involving data analysis and logistics, showing the ability to apply AI in real-world scenarios.
- The candidate has experience in research and development, which is crucial for staying updated with the latest technology trends.
- They have a degree from a reputable university, which indicates a strong educational background.

Areas for Improvement:
- The resume lacks specific achievements or metrics that could demonstrate the impact of the candidate's work. For example, it would be helpful to know if they led any successful AI projects or contributed to any significant innovations.
- There is no mention of leadership experience, which 

In [146]:
resume = """
in

 วิศวกรปัญญาประดิษฐ์

 ประสบการณ์การทำงาน
 วิศวกร ai ฝึกหัด
บริษัท เทคโซลูชันส์ จำกัด
enhanced team efficiency by lo% through a series of workshops on developing business applications
 raised annual promotions by 5% through the establishment ofa mentorship program for junior software
 engineers
พัฒนาโมเดล machine learning เพื่อพยากรณ์ความต้องการของลูกค้าและพฤติกรรมผู้บริโภค ช่วยไห้แคมเปญการตลาดมี
ประสิทธิภาพมากขึ้น
 ร่วมมือกับทีมวิเคราะห์ข้อมูลในการจัดการและเตรียมข้อมูลขนาดใหญ่เพื่อนำมาฝึกโมเดล
ออกแบบและพัฒนา dashboard สำหรับการวิเคราะห์ข้อมูลโดยใช้ power bi และ tableau เพื่อให้ทีมการตลาดสามารถ
ทำการตัดสินใจที่แม่นยำ
นักการตลาดดิจิทัล
บริษัท ฺมาร์เก็ตติ้ง พลัส


จัดการแคมเปญการตลาดดิจิทัลโดยใช้ข้อมูลเชิงลึกจากการวิเคราะห์ข้อมูลลูกค้า ซึงนำไปสู่การเพิ่มยอดขายออนไลน์ 15%
ใช้เทคนิคการวิเคราะห์ข้อมูลเพื่อระบุความสนใจและพฤติกรรมของลูกค้า เช่วยปรับกลยุทธ์การโฆษณาให้แม่นยยิ่งขึ้น
ศึกษาและเรียนรู้การใช้ ai เพื่อปรับปรุงกระบวนการทางการตลาด เช่น การสร้างเนื้อหาอัตโนมัติ และการวิเคราะห์พฤติกรรม
ลูกค้าด้วยโมเดล nlp (natural language processing)
โปรเจกต์ส่วนตัว
การพยากรณ์พฤติกรรมผู้บริโภคด้วย machine learning
พัฒนาโมเดลการพยากรณ์ด้วย machine learning เพื่อวิเคราะห์พฤติกรรมการซื้อสินค้าออนไลน์โดยใช้ข้อมูลจาก kaggle
ทำการ preprocessing ข้อมูลและใช้ scikit learn ในการฝึกโมเดลการจำแนกประเภท
การวิเคราะห์ความคิดเห็น ลูกค้า (sentiment analysis)
ศึกษาการประมวลผลภาษาธรรมชาติ (nlp) และพัฒนาโมเดลสำหรับวิเคราะห์ความคิดเห็นของลูกค้าบนโซเชียลมีเดีย โดยใช้
python และ hugging face transformers
 ประวัติการศึกษา
 ปริญญาตรีบริหารธุรกิจฺสาขาการตลาด
มหาวิทยาลัยธรรมศาสตร์
 ทักษะ
 ประกาศนียบัตรและการฝึกอบรม
การเขียนโปรแกรม python sol
 google data analytics professional
machine learning scikit
certificate
 coursera machine learning by
learn tensorflow
การวิเคราะห์ข้อมูล pandas
stanford university
 udacity ai programming with
 numpy tableau
 เครื่องมืออื่นๆ power bi
python nanodegree
 google analytics
"""
generated_answer = generate_answer(resume)
print(generated_answer)

Assistant: Evaluation:

Score: 5

Strengths:
- The candidate has a background in marketing, which could be beneficial for understanding customer behavior and applying AI to business problems.
- Experience in developing machine learning models for customer demand forecasting and behavior analysis, which is relevant to the job.
- Familiarity with data visualization tools like Power BI and Tableau, which are useful for data analysis and reporting.
- Personal projects that demonstrate practical experience with machine learning and NLP, such as sentiment analysis using Python and Hugging Face Transformers.

Areas for Improvement:
- Lack of direct experience in AI, machine learning, deep learning, and PyTorch, which are key requirements for the Senior AI Engineer role.
- The candidate's education is in Business Administration with a focus on marketing, not in a technical field like Computer Science, which could be a concern for a technical role.
- No clear evidence of leadership experience o

In [149]:
resume = """
วิศวกรปัญญาประดิษฐ์
ประสบการณ์การฝึกงาน
บัณฑิตจบใหม่สาขาวิทยาการคอมพิวเตอร์ทีมีความ
นักพัฒนาฝึกหัด ai
สนใจและทักษะด้าน ai และ machine learning ผ่าน
บริษัท ไอที โซลูชันส์ จำกัด
การทำโปรเจกต์ทั้งในการเรียนและโปรเจกต์ส่วนตัว มี
ประสบการณ์ในการพัฒนาโมเดล machine learning
ช่วยทีมพัฒนาโมเดล machine learning เพื่อพยากรณ์
และ deep learning ด้วย python และ
ยอดขายสินค้า
tensorflow มุ่งมันพัฒนาทักษะ ai เพื่อสร้างโซลูชัน
ทำการทดสอบและปรับปรุงโมเดลด้วยเทคนิค
ที่มีประสิทธิภาพ
 hyperparameter tuning
เรียนรู้และใช้เครื่องมือในการจัดการข้อมูลและการฝึก
โมเดล ai
ทักษะ
ภาษาโปรแกรม python c++
โปรเจกต์ในมหาวิทยาลัย
machine learning scikit learn
การวิเคราะห์ภาพใบหน้าเพื่อระบุอารมณ์
tensorflow keras
การจัดการข้อมูล sol pandas numpy
(emotion recognition)
เครื่องมืออื่นต git jupyter notebook
พัฒนาโมเดล deep learning ด้วย tensorflow เพือ
 จำแนกอารมณ์จากภาพใบหน้า
ใช้ชุดข้อมูล fer และฝึกโมเดล cmn เพือเพิ่ม

ความแม่นยำในการจำแนก

การวิเคราะห์ความเห็นบนโซเชียลมีเดีย
email
(sentiment analysis)
 linkedln
พัฒนาโมเดล deep learning ด้วย tensorflow เพือ
 จำแนกอารมณ์จากภาพใบหน้า
ใช้ชุดข้อมูล fer และฝึกโมเดล cnn เพื่อเพิ่ม
ความแม่นยำในการจำแนก
การศึกษา
 ประกาศนียบัตร
 ปริญญาตรีวิทยาศาสตร์ สาขาวิทยาการคอมพิวเตอร์
จบการศึกษา
มหาวิทยาลัยเกษตรศาสตร์
 ใช้ nlp และ machine learning เพื่อวิเคราะห์ความคิด
เห็นจากโซเชียลมีเดีย
 เกรดเฉลีย 3.70
 ใช้ python scikit learn และ nltk ในการพัฒนา
โปรเจกต์วิจัย ฺการพัฒนาโมเดล deep
โมเดลการจัดหมวดหมู่ความคิดเห็นเป็นบวกหรือลบ
 learning สำหรับการทำนายความต้องการของ
ลูกค้าในระบบอีคอมเมิร์ซ
กิจกรรมและความสนใจ
สมาชิกชมรมวิจัยและพัฒนา ai มหาวิทยาลัย
 เกษตรศาสตร์
สนใจการประยุกต์ใช้ ai ในด้านการพยากรณ์ทางธุรกิจ
และการวิเคราะห์ ข้อมูลเชิงลึก
"""
generated_answer = generate_answer(resume)
print(generated_answer)

Evaluation:

Score: 6
Strengths: 
1. The candidate has a solid educational background in computer science from a recognized university, which is a positive sign.
2. The candidate has a keen interest in AI and machine learning, which is evident from their participation in AI-related projects both in university and personal projects.
3. They have experience in developing machine learning models using Python and TensorFlow, which are essential skills for the position.
4. They have worked on projects that involve deep learning, NLP, and data analysis, which are relevant to the job requirements.

Areas for Improvement:
1. The resume is written in a mix of Thai and English, which may create confusion for international employers or HR personnel who are not fluent in Thai.
2. The candidate lacks significant industry experience, as they are a fresh graduate. The role requires experience in formulating and strategizing AI solutions, which the candidate has not demonstrated.
3. The candidate has 